In [1]:
# default_exp trackedfile
# hide
_FNAME='trackedfile'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
from tempfile import TemporaryDirectory

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [5]:
#export
import yaml

from dvcrecord.utils import maybe_yaml, write_yaml

class TrackedFile:
    def __init__(self, fpath, *args, **kwargs):
        self.fpath = fpath
        self.args = args
        self.kwargs = kwargs
        self.file_obj = None

    def __enter__(self):
        self.file_obj = open(self.fpath, *self.args, **self.kwargs)
        return self.file_obj

    def __exit__(self, *args):
        self.file_obj.close()

    @classmethod
    def from_previous_output(clz, stage, *args, pipeline_file=None, fpath=None, **kwargs):
        key = 'outs'
        pipeline_file = pipeline_file or  PIPELINE_FILE_DEFAULT
        with open(pipeline_file) as p:
            pipeline = yaml.safe_load(p)

        fnames = pipeline['stages'][stage][key]
        if fpath is not None:
            assert fpath in fnames
        fpath = fpath or fnames[0]
        return clz(fpath, *args, **kwargs)

    def render(self, as_yaml=False):
        return maybe_yaml(self.fpath, as_yaml)


In [3]:
def test_tf():
    with TemporaryDirectory() as tempdir:
        fpath = os.path.join(tempdir, 'test.txt')

        tf = TrackedFile(fpath, 'w')
        with tf as fileobj:
            fileobj.write('asdf')
            TESTCASE.assertEqual(tf.fpath, fpath)

        #the fileobj is just a normal file.  It can be read later
        with open(fpath) as readfile:
            TESTCASE.assertEqual('asdf', readfile.read())

    TESTCASE.assertEqual(tf.render(as_yaml=False), fpath)
test_tf()

In [6]:
def test_previous_output():
    dvc_pipe = {'stages':
                    {'prep':
                            {'cmd': 'python fake.py',
                             'outs': ['prep-out.txt']}
                            }
                    }

    with TemporaryDirectory() as tempdir:
        pipeline_file = write_yaml(dvc_pipe, folder=tempdir, fname='dvc.yaml')

        tf = TrackedFile.from_previous_output(stage='prep', pipeline_file=pipeline_file)
        TESTCASE.assertEqual(tf.render(), 'prep-out.txt')
test_previous_output()

In [ ]:
notebook2script(_nbpath)